In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp

In [0]:
cd "~/eckertlab/Mitra/mapping/split_parallel/collapsed/"

In [0]:
bam_dir = "."
analysis_dir = os.path.join(bam_dir, "samtools1.3")
if not os.path.exists(analysis_dir):
    os.makedirs(analysis_dir)
assert os.path.exists(analysis_dir)

In [0]:
bam_files = open("bams.txt").readlines()
bam_files = sorted([x.strip() for x in bam_files])
bam_files = [os.path.abspath(x) for x in bam_files]

In [0]:
len(bam_files) == 381

In [0]:
samtools = "/home/cfriedline/bin/samtools"
bcftools = "/home/cfriedline/bin/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_92/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

In [0]:
assembly = "/gpfs_fs/home/eckertlab/SugarPine_genome/pila.v1.0.scafSeq"

In [0]:
!$samtools faidx {assembly}

In [0]:
def create_ploidy_file(args):
    import os
    bam_files, analysis_dir = args
    ploidy_file = os.path.join(analysis_dir, "%s.ploidy" % "all")
    if not os.path.exists(ploidy_file):
        with open(ploidy_file, "w") as o:
            for b in bam_files:
                name = "%s" % os.path.basename(b).split("_")[0]
                ploidy = 2
                o.write("%s\t%d\n" % (name, ploidy))
    return ploidy_file

In [0]:
def call_snps(args):
    import socket, os, stopwatch
    print(socket.gethostname())
    timer = stopwatch.Timer()
    samtools, reference, bam_sorted, bcftools, raw_vcf, out_dir, contig_names = args 
    if not out_dir:
        out_dir = os.environ['TMPDIR']
    raw_vcf = os.path.join(out_dir, os.path.basename(raw_vcf))
    ploidy_file = create_ploidy_file((bam_sorted, out_dir))
    pileup = "%s mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR %s -Iuvf %s %s | %s call -f GP,GQ -S %s -vmO z -o %s" % (samtools, 
                                                                     contig_names, reference, 
                                                                     ' '.join(bam_sorted), 
                                                                     bcftools,
                                                                     ploidy_file,                                                                
                                                                     raw_vcf) 
    
    pileup = "%s mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR %s -Iuvf %s %s | %s call -f GP,GQ -vmO z -o %s" % (samtools, 
                                                                     contig_names, reference, 
                                                                     ' '.join(bam_sorted), 
                                                                     bcftools,                                          
                                                                     raw_vcf) 

    
    #print(pileup)
    #!$pileup
    timer.stop()
    return pileup, timer.elapsed

In [0]:
bams_by_size = !ls -Sr *mapped.bam
bams_by_size = [os.path.abspath(x) for x in bams_by_size]

In [0]:
args = [samtools, 
        assembly, 
        bam_files, 
        bcftools, 
        "samtools_1.3.vcf.gz", 
        '/home/cfriedline/eckertlab/Mitra/mapping/split_parallel/collapsed/samtools1.3/',
        ""]

In [0]:
pileup, elapsed = call_snps(args)

In [0]:
pileup

In [0]:
with open(os.path.join(analysis_dir, "run.sh"), "w") as o:
    o.write("#!/bin/bash\n")
    o.write("%s\n" % pileup)

## Run on SGE
```bash
cd ~/eckertlab/Mitra/mapping/split_parallel/collapsed/samtools1.3
qsub -cwd -N samtools -j n -o samtools.out -e samtools.err -V -q godel199@godel96 run.sh
```

In [0]:
with open(os.path.join(analysis_dir, "samples.txt"), "w") as o:
    for b in bam_files:
        o.write("%s\n" % b)